# Clustering 
Using the 3 first components of the PCA, and trying to cluster the observations based on these 3 dimensions
source: https://realpython.com/k-means-clustering-python/

In [1]:
#libraries imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

####  Setting up

In [2]:
df1 = pd.read_excel("pca_full_1.xlsx")

EOFError: 

In [ ]:
#to avoid reloading the dataset everytime I want to clear the Kernel
df = df1.copy()

In [ ]:
df.rename({"principalcomponent1": "C1", "principalcomponent2": "C2", "principalcomponent3": "C3", "principalcomponent4":"C4", "principalcomponent5":"C5"}, axis="columns", inplace=True)

In [ ]:
dflater = df[df["year"]!=2019]
df = df[df["year"]==2019]

## K-means with the 5 principal components

#### chosing the apropriate number of clusters - Elbow method

In [ ]:
dffeatures = df[['C1', 'C2','C3', 'C4', 'C5']]

In [ ]:
sc = StandardScaler()
dffeatures_Sc = sc.fit_transform(dffeatures)

In [ ]:
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"max_iter": 300,
"random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(dffeatures_Sc)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

3 or 6 could be good numbers- I chose 6

In [ ]:
kmeans = KMeans(
init="random",
n_clusters=6,
n_init=500,
max_iter=300,
random_state=42
)

In [ ]:
kmeans.fit(dffeatures_Sc)

In [ ]:
kmeans.cluster_centers_

based on the cluster centers, I can see what each cluster repreents (comparing the center for each component and each cluster)

In [ ]:
kmeans.labels_[:50]
#looking at the first 5 predicted labels ()

## K-means with 2 components

In [ ]:
dffeatures = df[['C1', 'C2']]

In [ ]:
sc = StandardScaler()
dffeatures_Sc = sc.fit_transform(dffeatures)

chosing the number of clusters when there are 2 components

In [ ]:
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"max_iter": 300,
"random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(dffeatures_Sc)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

clearly need 3 clusters, when 2 components are used

In [ ]:
kmeans = KMeans(
init="random",
n_clusters=3,
n_init=500,
max_iter=300,
random_state=42
)

In [ ]:
kmeans.fit(dffeatures_Sc)

In [ ]:
kmeans.cluster_centers_

plotting

In [ ]:
plt.scatter(dffeatures_Sc[:, 0], dffeatures_Sc[:, 1], c=kmeans.labels_)
plt.title("3 clusters 2 Components")
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

## K means with 3 components

In [ ]:
dffeatures = df[['C1', 'C2', "C3"]]

In [ ]:
sc = StandardScaler()
dffeatures_Sc = sc.fit_transform(dffeatures)

chosing the number of clusters

In [ ]:
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"max_iter": 300,
"random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(dffeatures_Sc)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

The optimal number of clusters is 4. 

K-means with the optimal number of clusters

In [ ]:
kmeans = KMeans(
init="random",
n_clusters=4,
n_init=500,
max_iter=300,
random_state=42
)

In [ ]:
kmeans.fit(dffeatures_Sc)

In [ ]:
centers = kmeans.cluster_centers_

#### plotting

In [ ]:
dffeatures_Sc = pd.DataFrame(dffeatures_Sc)
dfplot = pd.DataFrame(kmeans.labels_)
dfplot.rename(columns={0: "Cluster"}, inplace=True)
dfplot =  pd.concat([dffeatures_Sc,dfplot], axis=1)
dfplot.rename(columns={0: "C1", 1: "C2", 2:"C3"}, inplace=True)

3D plot 

In [ ]:
import matplotlib.pyplot as plt
import random

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter(dfplot[dfplot["Cluster"]==0]["C1"], dfplot[dfplot["Cluster"]==0]["C2"],dfplot[dfplot["Cluster"]==0]["C3"], c="lightseagreen", label="Cluster 1")
ax.scatter(dfplot[dfplot["Cluster"]==1]["C1"], dfplot[dfplot["Cluster"]==1]["C2"],dfplot[dfplot["Cluster"]==1]["C3"], c="teal", label="Cluster 2")
ax.scatter(dfplot[dfplot["Cluster"]==2]["C1"], dfplot[dfplot["Cluster"]==2]["C2"],dfplot[dfplot["Cluster"]==2]["C3"], c="palevioletred", label="Cluster 3")
ax.scatter(dfplot[dfplot["Cluster"]==3]["C1"], dfplot[dfplot["Cluster"]==3]["C2"],dfplot[dfplot["Cluster"]==3]["C3"], c="palegoldenrod", label="Cluster 4")
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')


#ax.view_init(-160, 140) # just testing soma angles
#ax.view_init(-160, 20) # see well the yellow cluster
ax.view_init(-140, 120) #ssee well clusters 1 and 4
plt.legend()
plt.show()

##  understanding what these clusters are

Do to so: computing the means of each variable for each cluster, and the mean of the variable itself (unclustered). Then computing the percentage difference between the mean of the variable and the mean of the cluster. Excluding the variables that have no variance (mean smaller than 0.05 or larger than 0.95 but smaller than 1). Then for each cluster, listing the top 10 variables for which the mean of thecluster (in %) is very different from the mean of the variable.

#### Computing the means per cluster for each variable

In [ ]:
clusters = pd.DataFrame(kmeans.labels_) #save the cluster number as a dataframe
clusters = clusters.rename(index=dict(zip(clusters.index.tolist(),dffeatures.index.tolist()))) #reindex this dataframe
df = pd.concat([df, clusters], axis=1).reindex(dffeatures.index) #add cluster numbers to original dataframe
df.rename({0:"cluster"}, axis=1, inplace=True)

In [ ]:
dfmeans = df.drop(["year","X_uuid", "governorate", "district", "weight", "population_group"], axis=1)

In [ ]:
dfmeans = dfmeans.groupby("cluster").mean()

In [ ]:
dfmeans.rename({0:"Cluster1mean", 1:"Cluster2mean", 2:"Cluster3mean", 3:"Cluster4mean"}, inplace=True)
dfmeans = dfmeans.T

#### Computing the means of each variable

In [ ]:
#importing the original dataset to compute the average of each variable
df1x = pd.read_excel("final_clean.xlsx")

In [ ]:
dfx = df1.copy()
dfx = dfx[dfx["year"]==2019] #selecting only 2019

In [ ]:
dfx.rename({'principalcomponent1':"C1", 'principalcomponent2':"C2", 'principalcomponent3':"C3",
       'principalcomponent4':"C4", 'principalcomponent5':"C5"}, axis=1, inplace=True)
dfx = dfx.drop(["year","X_uuid", "governorate", "district", "weight", "population_group"], axis=1)

In [ ]:
dfxmeans = pd.DataFrame(dfx.mean()) # computing the mean

#### Combining the datasets with all means

In [ ]:
dfmeansfinal = pd.concat([dfmeans,dfxmeans],axis=1)

In [ ]:
dfmeansfinal.rename(columns={ dfmeansfinal.columns[4]: "Varmean" }, inplace=True) 

In [ ]:
dfmeansfinal.drop(["C1", "C2", "C3", "C4", "C5", "quintile"], inplace=True)

In [ ]:
dfmeansfinal["C1perc"] = ((dfmeansfinal["Cluster1mean"]-dfmeansfinal["Varmean"])/dfmeansfinal["Varmean"])*100
dfmeansfinal["C2perc"] = ((dfmeansfinal["Cluster2mean"]-dfmeansfinal["Varmean"])/dfmeansfinal["Varmean"])*100
dfmeansfinal["C3perc"] = ((dfmeansfinal["Cluster3mean"]-dfmeansfinal["Varmean"])/dfmeansfinal["Varmean"])*100
dfmeansfinal["C4perc"] = ((dfmeansfinal["Cluster4mean"]-dfmeansfinal["Varmean"])/dfmeansfinal["Varmean"])*100

In [ ]:
dfmeansfinal.to_excel("finalmeanspercluster.xlsx", index=True)  

#### Dropping all variables for which the average value of the variable is less than 0.05 or more than 0.95 but less than 1 (they are all dummies with not enough variance)

In [ ]:
dfmeansfinalreduced = dfmeansfinal.copy()

In [ ]:
todrop0 = dfmeansfinalreduced[(dfmeansfinalreduced["Varmean"]<0.05) | ((dfmeansfinalreduced["Varmean"]>0.95) & (dfmeansfinalreduced["Varmean"]<=1))].index
for col in todrop0:
    dfmeansfinalreduced.drop(col, axis=0, inplace=True)

In [ ]:
#dropping all receivd_aid_ columns because have almost no variance
dfmeansfinalreduced.loc[~dfmeansfinalreduced.index.str.startswith("received_aid_"),:]

In [ ]:
#dropping all received_aid
dfmeansfinalreduced = dfmeansfinalreduced.loc[~dfmeansfinalreduced.index.str.startswith("received_aid_"), :]

In [ ]:
dfmeansfinalreduced

In [ ]:
#absolute values
dfmeansfinalreduced["C1percabs"] = abs(dfmeansfinalreduced["C1perc"])
dfmeansfinalreduced["C2percabs"] = abs(dfmeansfinalreduced["C2perc"])
dfmeansfinalreduced["C3percabs"] = abs(dfmeansfinalreduced["C3perc"])
dfmeansfinalreduced["C4percabs"] = abs(dfmeansfinalreduced["C4perc"])

#### Cluster 1 - main differences

Classical HH structure (not single headed, and with children). These households are better off than average in terms of shelter and nutrition. Close to average for most indicators. more children working than average, children are not married

In [ ]:
dfmeansfinalreduced.nlargest(10, ['C1percabs']).C1perc

#### Cluster 2 - main differences

This is the poorest cluster: high food_exp_share, high proportion of ppl living in tents and critical shelters, high proportion of ppl having received aid. But children attend school more than average. No single-headed households in this cluster

In [ ]:
dfmeansfinalreduced.nlargest(10, ['C2percabs']).C2perc

#### Cluster 3- main differences

Poor families with many disabled HH members. Children dont go to school and work instead. Many parents are unemployed. Malnourished households (borderline FCS)

In [ ]:
dfmeansfinalreduced.nlargest(10, ['C3percabs']).C3perc

#### Cluster 4 - main differences

These HH have almostno children at all (99% have no child). Wealthier households than average (dont receive aid,...). More single headed HH than average

In [ ]:
dfmeansfinalreduced.nlargest(10, ['C4percabs']).C4perc

now columns C1perc and others show the percentage variation compared to the mean for each variable and each cluster (in cluster 1, the mean of num_hh_member is 5.74e-02 times larger than the mean of the variable). need to flag the ones that stand out most, and that could hence represent the cluster (givemeaning to the cluster)

### In addition to doing this, do  a ttest or a chi-sq test to check if the variables are different from the mean of the dataset

## Using the same cluster centers to assign a cluster to the observations in years 2020 and 2021  = NEXT STEP

In [ ]:
dflaterfeatures = dflater[['C1', 'C2','C3']]
dflaterfeatures_Sc = sc.transform(dflaterfeatures)

In [ ]:
kmeans = KMeans(
init=centers,
n_clusters=4,
n_init=1,
max_iter=1
)

In [ ]:
kmeans.fit(dflaterfeatures_Sc)

In [ ]:
centers

In [ ]:
kmeans.cluster_centers_